In [ ]:
##################################### Wide Camera + Laser Control (1920x1440 pixels) #####################################
import cv2
import numpy as np
import math
import socket
import threading
import time
import sys
from flask import Flask, jsonify, request, Response

# For Jupyter/IPython compatibility
try:
    sys.stdout.reconfigure(line_buffering=True)
    sys.stderr.reconfigure(line_buffering=True)
except AttributeError:
    pass

# --- DAC/Voltage Configuration ---
DAC_MAX = 1920
AO_RANGE = 5.0

# --- Network Configuration ---
HOST = '127.0.0.1'
PORT = 65432
daq_socket = None

# Web server for tablet
TABLET_SERVER_PORT = 8081
SERVER_IP = '0.0.0.0'

# --- Rotation Configuration ---
ROTATION_ANGLE_DEG = 0  # Rotation angle in degrees
ROTATION_ANGLE_RAD = np.radians(ROTATION_ANGLE_DEG)
ROTATION_CENTER_X = 960  # Center of 1920x1440
ROTATION_CENTER_Y = 720

# --- Drawing Configuration ---
MIN_DIST = 2              
MIN_JUMP_DIST = 10        
CLOSED_THRESHOLD = 100    
AREA_STEP = 10
SINGLE_POINT_THRESHOLD = 20

# --- Dark Spot Detection Configuration ---
GLOBAL_GRAYSCALE_THRESHOLD = 80
WINDOW_NAME = "Laser Controller"
MIN_AREA = 500
MAX_AREA = 500000

# --- Galvo Configuration (adjusted for 1920x1440) ---
HOME_X = 960
HOME_Y = 720
GALVO_MIN_X = 125
GALVO_MAX_X = 1750
GALVO_MIN_Y = 0
GALVO_MAX_Y = 1333
CROSS_SIZE = 10

# --- Global State ---
contours = []
finished_polys = []
last_active_contour = None
detected_contours = []
current = []
drawing = False

# Video Streaming State
video_lock = threading.Lock()
output_frame = None

# Touch state
touch_active = False
touch_start_pos = None
touch_current_path = []

# --- CALIBRATION STATE ---
CALIBRATION_PIXEL_POINTS = []
CALIBRATION_DATA_MAP = []
MANUAL_CALIBRATION_MODE = False

contour_lock = threading.Lock()
socket_lock = threading.Lock()

# Global reference to camera capture
cap = None
running = True

# --- Area preview (density-based) ---
area_preview_points = []
area_preview_lock = threading.Lock()

# --- Flask App for Tablet Interface ---
app = Flask(__name__)

@app.before_request
def log_request_info():
    print(f"\n>>> INCOMING REQUEST: {request.method} {request.path}", flush=True)

# HTML Interface
TABLET_HTML = """
<!DOCTYPE html>
<html>
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0, user-scalable=no">
<meta http-equiv="Cache-Control" content="no-cache, no-store, must-revalidate">
<meta http-equiv="Pragma" content="no-cache">
<meta http-equiv="Expires" content="0">
<style>
body {
    margin: 0;
    background: #000;
    color: #0f0;
    font-family: monospace;
    overflow: hidden;
}
#touchpad {
    position: absolute;
    top: 0;
    left: 0;
    width: 100%;
    height: 100%;
    z-index: 10;
    touch-action: none;
}
#video-background {
    position: absolute;
    width: 100%;
    height: 100%;
    object-fit: contain;
}
.sidebar {
    position: absolute;
    right: 20px;
    top: 20px;
    bottom: 20px;
    display: flex;
    flex-direction: column;
    gap: 15px;
    z-index: 25;
}
.btn {
    width: 120px;
    height: 80px;
    background: rgba(0,0,0,0.85);
    border: 3px solid #0f0;
    display: flex;
    align-items: center;
    justify-content: center;
    border-radius: 15px;
    font-weight: bold;
    font-size: 24px;
    cursor: pointer;
    color: #0f0;
    transition: all 0.2s;
}
.btn:active {
    transform: scale(0.95);
    background: rgba(0,255,0,0.2);
}
.btn-exit {
    border-color: #ff0;
    color: #ff0;
}
.btn-exit:active {
    background: rgba(255,255,0,0.2);
}
.btn-lase {
    border-color: #f00;
    color: #f00;
    font-size: 22px;
}
.btn-lase:active {
    background: rgba(255,0,0,0.2);
}
.slider-container {
    width: 120px;
    background: rgba(0,0,0,0.85);
    border: 3px solid #0f0;
    border-radius: 15px;
    display: flex;
    flex-direction: column;
    align-items: center;
    justify-content: center;
    padding: 15px 0;
    gap: 10px;
}
.slider-label {
    font-size: 18px;
    color: #0f0;
    font-weight: bold;
}
input[type=range][orient=vertical] {
    -webkit-appearance: slider-vertical;
    width: 40px;
    height: 200px;
    cursor: pointer;
}
.slider-value {
    font-size: 28px;
    color: #fff;
    font-weight: bold;
}
</style>
</head>
<body>
<img id="video-background" src="/video_feed">
<div id="touchpad"></div>
<div class="sidebar">
    <div class="btn btn-lase" onclick="cmd('lasing')">LASING</div>
    <div class="slider-container">
        <div class="slider-label">DENSITY</div>
        <input type="range" min="5" max="100" step="5" value="10"
               orient="vertical" oninput="updateDensity(this.value)">
        <div class="slider-value" id="dval">10</div>
    </div>
    <div class="slider-container">
        <div class="slider-label">THRESH</div>
        <input type="range" min="0" max="255" value="80" 
               orient="vertical" oninput="updateThresh(this.value)">
        <div class="slider-value" id="tval">80</div>
    </div>
    <div class="btn" onclick="cmd('calibrate')">CALIB</div>
    <div class="btn" onclick="cmd('home')">HOME</div>
    <div class="btn" onclick="cmd('clear')">CLEAR</div>
    <div class="btn btn-exit" onclick="cmd('exit')">EXIT</div>
</div>
<script>
function cmd(c) {
    fetch('/command', {method:'POST', headers:{'Content-Type':'application/json'}, body:JSON.stringify({command:c})})
    .then(response => response.json())
    .then(data => console.log("Response:", data))
    .catch(error => console.error("Error:", error));
}
function updateThresh(v) {
    document.getElementById('tval').innerText = v;
    fetch('/update_threshold', {method:'POST', headers:{'Content-Type':'application/json'}, body:JSON.stringify({threshold:parseInt(v)})});
}
function updateDensity(v) {
    document.getElementById('dval').innerText = v;
    fetch('/update_density', {method:'POST', headers:{'Content-Type':'application/json'}, body:JSON.stringify({density:parseInt(v)})});
}
const tp = document.getElementById('touchpad');
function send(t, cx, cy) {
    const r = tp.getBoundingClientRect();
    const x = Math.round(((cx - r.left)/r.width)*1920);
    const y = Math.round(((cy - r.top)/r.height)*1440);
    fetch('/touch_input', {method:'POST', headers:{'Content-Type':'application/json'}, body:JSON.stringify({touching:t, x:x, y:y})});
}
tp.addEventListener('touchstart', (e)=>{ e.preventDefault(); send(true, e.touches[0].clientX, e.touches[0].clientY); });
tp.addEventListener('touchmove', (e)=>{ e.preventDefault(); send(true, e.touches[0].clientX, e.touches[0].clientY); });
tp.addEventListener('touchend', ()=>{ send(false, 0, 0); });
</script>
</body>
</html>
"""

def generate_web_stream():
    global output_frame
    last_frame_time = 0
    fps_limit = 1/20
    
    while True:
        current_time = time.time()
        if current_time - last_frame_time < fps_limit:
            time.sleep(0.03)
            continue
        
        with video_lock:
            if output_frame is None:
                continue
            encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), 30]
            flag, encodedImage = cv2.imencode(".jpg", output_frame, encode_param)
            if not flag:
                continue
        
        last_frame_time = current_time
        yield(b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' + bytearray(encodedImage) + b'\r\n')

@app.route('/')
def tablet_interface():
    return TABLET_HTML

@app.route("/video_feed")
def video_feed():
    return Response(generate_web_stream(), mimetype="multipart/x-mixed-replace; boundary=frame")

@app.route('/command', methods=['POST'])
def handle_ui_command():
    global running, CALIBRATION_PIXEL_POINTS, detected_contours, daq_socket
    global finished_polys, last_active_contour, current, contours, AREA_STEP, area_preview_points
    
    data = request.json
    cmd = data.get('command')
    print(f"\n[COMMAND] {cmd.upper()}", flush=True)
    
    if cmd == 'lasing':
        if last_active_contour is not None:
            print(f"[LASING] Starting laser operation...", flush=True)
            with area_preview_lock:
                area_pts = list(area_preview_points)
            # Use the new LASING specific sender
            send_lasing_to_server(area_pts, daq_socket)
        else:
            print("[LASING] No active contour selected", flush=True)
    
    elif cmd == 'home':
        if daq_socket:
            with socket_lock:
                rotated = rotate_point(HOME_X, HOME_Y, ROTATION_ANGLE_RAD, ROTATION_CENTER_X, ROTATION_CENTER_Y)
                daq_socket.sendall(f"{int(rotated[0])},{int(rotated[1])}\n".encode('ascii'))
                print(f"[HOME] Sent: {rotated}", flush=True)
    
    elif cmd == 'clear':
        with contour_lock:
            finished_polys.clear()
            contours.clear()
            current = []
            last_active_contour = None
        with area_preview_lock:
            area_preview_points.clear()
        print("[CLEAR] All cleared including preview", flush=True)
    
    elif cmd == 'calibrate':
        threading.Thread(target=calibrate_galvo_camera, daemon=True).start()
    
    elif cmd == 'exit':
        running = False
    
    return jsonify({'status': 'ok'})

@app.route('/touch_input', methods=['POST'])
def receive_touch():
    data = request.json
    touching, x, y = data.get('touching'), data.get('x'), data.get('y')
    
    if touching:
        if not touch_active:
            handle_touch_down(x, y)
        else:
            handle_touch_move(x, y)
    elif touch_active:
        handle_touch_up(x, y)
    
    return jsonify({'status': 'ok'})

@app.route('/update_threshold', methods=['POST'])
def update_threshold():
    global GLOBAL_GRAYSCALE_THRESHOLD
    val = request.json.get('threshold', 80)
    GLOBAL_GRAYSCALE_THRESHOLD = int(val)
    return jsonify({'status': 'ok'})


@app.route('/update_density', methods=['POST'])
def update_density():
    global AREA_STEP
    data = request.json
    val = data.get('density', 10)
    
    # Update the global step
    AREA_STEP = int(np.clip(val, 5, 100))
    
    # Explicitly trigger the recalculation of the coordinates
    update_area_preview()
    
    print(f"[DENSITY] Variable updated to: {AREA_STEP}", flush=True)
    return jsonify({'status': 'ok', 'area_step': AREA_STEP})


def run_flask_server():
    print("\n" + "="*60, flush=True)
    print("FLASK SERVER STARTING", flush=True)
    print(f"Server: http://{SERVER_IP}:{TABLET_SERVER_PORT}", flush=True)
    print(f"Rotation: {ROTATION_ANGLE_DEG} deg around ({ROTATION_CENTER_X}, {ROTATION_CENTER_Y})", flush=True)
    print("="*60 + "\n", flush=True)
    app.run(host=SERVER_IP, port=TABLET_SERVER_PORT, debug=False, threaded=True)

# --------------------------------------------
# ROTATION TRANSFORMATION
# --------------------------------------------
def rotate_point(x, y, angle_rad, cx, cy):
    x_shifted = x - cx
    y_shifted = y - cy
    
    cos_a = np.cos(angle_rad)
    sin_a = np.sin(angle_rad)
    x_rot = x_shifted * cos_a - y_shifted * sin_a
    y_rot = x_shifted * sin_a + y_shifted * cos_a
    
    x_final = x_rot + cx
    y_final = y_rot + cy
    
    return (x_final, y_final)

def rotate_points_batch(points, angle_rad, cx, cy):
    return [rotate_point(x, y, angle_rad, cx, cy) for x, y in points]

# --------------------------------------------
# HSV LASER DETECTION
# --------------------------------------------
def find_laser_spot_hsv(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    l1, u1 = np.array([0, 50, 50]), np.array([10, 255, 255])
    l2, u2 = np.array([160, 50, 50]), np.array([180, 255, 255])
    mask = cv2.addWeighted(cv2.inRange(hsv, l1, u1), 1.0, cv2.inRange(hsv, l2, u2), 1.0, 0)
    cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if cnts:
        c = max(cnts, key=cv2.contourArea)
        if cv2.contourArea(c) > 2:
            M = cv2.moments(c)
            if M["m00"] != 0:
                return (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
    return None

# --------------------------------------------
# AREA & BOUNDARY PROCESSING
# --------------------------------------------
def get_area_points(polygon, step):
    """Generate grid points inside polygon using the provided step size"""
    pts_to_send = []
    # Ensure polygon is the correct shape for OpenCV
    if not isinstance(polygon, np.ndarray):
        polygon = np.array(polygon, dtype=np.int32)
        
    x, y, w, h = cv2.boundingRect(polygon)
    s = int(step) # Force integer step
    
    for j in range(y, y + h, s):
        for i in range(x, x + w, s):
            # The 'False' parameter means we just want to know if it's inside (True/False)
            if cv2.pointPolygonTest(polygon, (float(i), float(j)), False) >= 0:
                pts_to_send.append((i, j))
    return pts_to_send


def update_area_preview():
    """Recompute area fill preview using latest AREA_STEP"""
    global area_preview_points, last_active_contour, AREA_STEP

    if last_active_contour is None:
        with area_preview_lock:
            area_preview_points = []
        return

    # Generate points into a temporary list first
    new_pts = get_area_points(last_active_contour, AREA_STEP)
    
    # Swap them into the global list under lock
    with area_preview_lock:
        area_preview_points = new_pts
        
    print(f"[PREVIEW] {len(area_preview_points)} dots at step {AREA_STEP}", flush=True)


def distance(p1, p2):
    return math.hypot(int(p1[0]) - int(p2[0]), int(p1[1]) - int(p2[1]))

def get_sampled_contour(contour, min_jump_dist):
    points = np.array(contour).reshape(-1, 2)
    if not points.any():
        return []
    
    sampled = [tuple(points[0])]
    last_point = points[0]
    
    for pt in points[1:]:
        if distance(last_point, pt) >= min_jump_dist:
            sampled.append(tuple(pt))
            last_point = pt
    
    return sampled

def send_batch_to_server(points_list, daq_socket):
    if not daq_socket or not points_list:
        return
    
    rotated_points = rotate_points_batch(points_list, ROTATION_ANGLE_RAD, ROTATION_CENTER_X, ROTATION_CENTER_Y)
    
    with socket_lock:
        try:
            daq_socket.sendall(b"BATCH_START\n")
            data_string = ";".join(["{0},{1}".format(int(x), int(y)) for x, y in rotated_points])
            daq_socket.sendall(data_string.encode('ascii') + b"\n")
            print(f"[BATCH] Sent {len(rotated_points)} rotated points", flush=True)
        except Exception as e:
            print(f"Batch error: {e}")

def find_dark_spots(frame):
    global GLOBAL_GRAYSCALE_THRESHOLD, detected_contours
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(gray, GLOBAL_GRAYSCALE_THRESHOLD, 255, cv2.THRESH_BINARY_INV)
    kernel = np.ones((3, 3), np.uint8)
    mask = cv2.dilate(cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel), kernel, iterations=1)
    detected, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    detected_contours = [c for c in detected if MIN_AREA < cv2.contourArea(c) < MAX_AREA]
    return detected_contours

def send_lasing_to_server(points_list, daq_socket):
    """Sends points with a special header to trigger the 5V laser signal"""
    if not daq_socket or not points_list:
        return
    
    rotated_points = rotate_points_batch(points_list, ROTATION_ANGLE_RAD, ROTATION_CENTER_X, ROTATION_CENTER_Y)
    
    with socket_lock:
        try:
            # Special header for 5V trigger
            daq_socket.sendall(b"LASING_START\n")
            
            data_string = ";".join(["{0},{1}".format(int(x), int(y)) for x, y in rotated_points])
            daq_socket.sendall(data_string.encode('ascii') + b"\n")
            
            # Special footer to ensure laser is turned off immediately after
            daq_socket.sendall(b"LASING_END\n")
            
            print(f"[LASING] Sent {len(rotated_points)} points with trigger headers", flush=True)
        except Exception as e:
            print(f"Lasing socket error: {e}")
# --------------------------------------------
# TOUCH INPUT HANDLER
# --------------------------------------------
def calculate_path_length(points):
    if len(points) < 2:
        return 0
    return sum(distance(points[i-1], points[i]) for i in range(1, len(points)))

def handle_touch_down(x, y):
    global drawing, current, touch_active, touch_start_pos, touch_current_path
    global finished_polys, last_active_contour, detected_contours
    
    if MANUAL_CALIBRATION_MODE:
        return
    
    pt = (int(x), int(y))
    
    for detected_c in detected_contours:
        if cv2.pointPolygonTest(detected_c, pt, False) >= 0:
            poly_arr = np.array(detected_c, dtype=np.int32).reshape(-1, 2)
            finished_polys.append(poly_arr)
            last_active_contour = poly_arr
            update_area_preview()
            send_batch_to_server(get_sampled_contour(poly_arr, MIN_JUMP_DIST), daq_socket)
            return
    
    touch_active = True
    touch_start_pos, touch_current_path = pt, [pt]
    drawing = True
    current = []
    
    with contour_lock:
        contours.clear()
        contours.append(current)
    current.append(pt)

def handle_touch_move(x, y):
    global current, touch_current_path
    
    if not touch_active or MANUAL_CALIBRATION_MODE:
        return
    
    pt = (int(x), int(y))
    touch_current_path.append(pt)
    
    if not current or distance(current[-1], pt) >= MIN_DIST:
        with contour_lock:
            current.append(pt)

def handle_touch_up(x, y):
    global drawing, current, touch_active, touch_start_pos, touch_current_path
    global finished_polys, last_active_contour
    
    if not touch_active or MANUAL_CALIBRATION_MODE:
        return
    
    touch_active = False
    drawing = False
    path_len = calculate_path_length(touch_current_path)
    
    if path_len < SINGLE_POINT_THRESHOLD:
        if daq_socket:
            with socket_lock:
                rotated = rotate_point(touch_start_pos[0], touch_start_pos[1], ROTATION_ANGLE_RAD, ROTATION_CENTER_X, ROTATION_CENTER_Y)
                daq_socket.sendall(f"{int(rotated[0])},{int(rotated[1])}\n".encode('ascii'))
    elif len(current) > 5:
        if distance(current[0], current[-1]) < CLOSED_THRESHOLD:
            current.append(current[0])
            poly_arr = np.array(current, dtype=np.int32)
            finished_polys.append(poly_arr)
            last_active_contour = poly_arr
            update_area_preview()
            send_batch_to_server(get_sampled_contour(poly_arr, MIN_JUMP_DIST), daq_socket)
        else:
            send_batch_to_server(get_sampled_contour(current, MIN_JUMP_DIST), daq_socket)
    
    current = []

# --------------------------------------------
# CALIBRATION
# --------------------------------------------
def calibrate_galvo_camera():
    global daq_socket, MANUAL_CALIBRATION_MODE, CALIBRATION_DATA_MAP, CALIBRATION_PIXEL_POINTS, cap
    
    if not daq_socket or cap is None:
        print("ERROR: DAQ or camera not initialized")
        return False
    
    MANUAL_CALIBRATION_MODE = True
    CALIBRATION_DATA_MAP.clear()
    CALIBRATION_PIXEL_POINTS.clear()
    
    with socket_lock:
        try:
            # 1. Notify Start
            daq_socket.sendall(b"CALIBRATION_START\n")
            time.sleep(0.5)

            cx, cy = (GALVO_MAX_X + GALVO_MIN_X)//2, (GALVO_MAX_Y + GALVO_MIN_Y)//2
            xs = np.linspace(GALVO_MIN_X, GALVO_MAX_X, CROSS_SIZE, dtype=np.int32)
            ys = np.linspace(GALVO_MIN_Y, GALVO_MAX_Y, CROSS_SIZE, dtype=np.int32)
            gal_pts = [(cx, int(y)) for y in ys] + [(int(x), cy) for x in xs if int(x) != cx]
            
            # 2. Project Cross and Capture
            for x_dac, y_dac in gal_pts:
                if not MANUAL_CALIBRATION_MODE: break
                
                rotated = rotate_point(x_dac, y_dac, ROTATION_ANGLE_RAD, ROTATION_CENTER_X, ROTATION_CENTER_Y)
                daq_socket.sendall(f"{int(rotated[0])},{int(rotated[1])}\n".encode('ascii'))
                
                # Wait for mirrors to settle and camera to catch up
                time.sleep(0.3) 
                
                found_pt = None
                start_search = time.time()
                while (time.time() - start_search) < 1.5:
                    ret, frame = cap.read()
                    if not ret: break
                    
                    spot = find_laser_spot_hsv(frame)
                    if spot:
                        found_pt = spot
                        CALIBRATION_PIXEL_POINTS.append(spot)
                        break
                
                if found_pt:
                    CALIBRATION_DATA_MAP.append(found_pt)
                    print(f"Captured: DAC({x_dac},{y_dac}) -> Pixel{found_pt}")

            # 3. Dump results to Server
            print(f"Sending {len(CALIBRATION_DATA_MAP)} points to server...")
            daq_socket.sendall(b"PIXEL_DUMP_START\n")
            time.sleep(0.1)
            
            for px, py in CALIBRATION_DATA_MAP:
                msg = "{0},{1}\n".format(int(px), int(py))
                daq_socket.sendall(msg.encode('ascii'))
            
            daq_socket.sendall(b"PIXEL_DUMP_END\n")
            print("Calibration stream finished successfully")

        except Exception as e:
            print(f"Calibration Socket Error: {e}")
    
    MANUAL_CALIBRATION_MODE = False
    return True

# -------------------------------------------- 
# MAIN EXECUTION
# --------------------------------------------
try:
    daq_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    daq_socket.connect((HOST, PORT))
    print("Connected to DAQ server")
except:
    daq_socket = None
    print("Warning: Could not connect to DAQ server")

flask_thread = threading.Thread(target=run_flask_server, daemon=True)
flask_thread.start()

cap = cv2.VideoCapture(1)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1440)

cv2.namedWindow(WINDOW_NAME, cv2.WINDOW_NORMAL)

while running:
    ret, frame = cap.read()
    if not ret:
        break

    if not MANUAL_CALIBRATION_MODE:
        detected_contours = find_dark_spots(frame)
        overlay = frame.copy()
        
        for poly in finished_polys:
            cv2.fillPoly(overlay, [poly], (0, 255, 0))
            cv2.polylines(frame, [poly], True, (0, 255, 0), 2)
        
        cv2.addWeighted(overlay, 0.5, frame, 0.5, 0, frame)

        with contour_lock:
            for cnt in contours:
                if len(cnt) >= 2:
                    pts = np.array(cnt, np.int32).reshape((-1, 1, 2))
                    cv2.polylines(frame, [pts], False, (255, 0, 255), 3)

        for pt in CALIBRATION_PIXEL_POINTS:
            cv2.drawMarker(frame, pt, (255, 0, 0), cv2.MARKER_CROSS, 20, 3)
        
        cv2.drawContours(frame, detected_contours, -1, (0, 0, 255), 2)
    
    if MANUAL_CALIBRATION_MODE:
        for pt in CALIBRATION_PIXEL_POINTS:
            cv2.drawMarker(frame, pt, (255, 0, 0), cv2.MARKER_CROSS, 20, 3)
    
    with area_preview_lock:
            # Create a local copy to iterate over so the list can change in the background
            current_preview = list(area_preview_points)
        
    for (px, py) in current_preview:
            # Use a smaller radius for high density to avoid a solid red blob
        radius = 5 
        cv2.circle(frame, (int(px), int(py)), radius, (0, 0, 255), -1)

    with video_lock:
        output_frame = frame.copy()
    
    cv2.imshow(WINDOW_NAME, frame)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'): 
        running = False
    elif key == ord('c'): 
        with contour_lock:
            finished_polys.clear()
            contours.clear()
            last_active_contour = None
        with area_preview_lock:
            area_preview_points.clear()
        print("Cleared")
    elif key == ord('a'): 
        threading.Thread(target=calibrate_galvo_camera, daemon=True).start()
    elif key == ord('l'):
        if last_active_contour is not None:
            with area_preview_lock:
                area_pts = list(area_preview_points)
            send_batch_to_server(area_pts, daq_socket)

if daq_socket:
    daq_socket.close()
cap.release()
cv2.destroyAllWindows()

Connected to DAQ server

FLASK SERVER STARTING
Server: http://0.0.0.0:8081
Rotation: 0 deg around (960, 720)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8081
 * Running on http://192.168.51.107:8081
Press CTRL+C to quit


In [ ]:
##################################### Wide Camera + Laser Control (1920x1440 with ELO Touch) #####################################
#compatible with (Server)_daq_listener_wide_camera_LASING#############################################################

import cv2
import numpy as np
import math
import socket
import threading
import time
import sys
from flask import Flask, jsonify, request, Response

# For Jupyter/IPython compatibility
try:
    sys.stdout.reconfigure(line_buffering=True)
    sys.stderr.reconfigure(line_buffering=True)
except AttributeError:
    # Running in Jupyter/IPython - output is already unbuffered
    pass

aiming_thread_active = False
aiming_contour = None
aiming_lock = threading.Lock()

# --- DAC/Voltage Configuration ---
DAC_MAX = 1920
AO_RANGE = 5.0
LASER_ON_TIME = 50  # Default 50ms

# --- Network Configuration ---
HOST = '127.0.0.1'
PORT = 65432
daq_socket = None

# Web server for tablet
TABLET_SERVER_PORT = 8081
SERVER_IP = '0.0.0.0'  # Bind to all network interfaces

# --- Drawing Configuration ---
MIN_DIST = 10             
MIN_JUMP_DIST = 10        
CLOSED_THRESHOLD = 100    
AREA_STEP = 10  # This will be updated by the density slider
SINGLE_POINT_THRESHOLD = 20

# --- Dark Spot Detection Configuration ---
GLOBAL_GRAYSCALE_THRESHOLD = 80
WINDOW_NAME = "Laser Controller"
MIN_AREA = 500
MAX_AREA = 500000

# --- Galvo Configuration (adjusted for 1920x1440) ---
HOME_X = 960              # Center X for 1920x1440
HOME_Y = 720              # Center Y for 1920x1440
GALVO_MIN_X = 125         # Scaled from 250
GALVO_MAX_X = 1750        # Scaled from 3500
GALVO_MIN_Y = 0
GALVO_MAX_Y = 1333        # Scaled from 2000
CROSS_SIZE = 10           

# --- Global State ---
contours = []             
finished_polys = []       
last_active_contour = None 
detected_contours = []
current = []
drawing = False 

# Video Streaming State
video_lock = threading.Lock()
output_frame = None

# Touch state
touch_active = False
touch_start_pos = None
touch_current_path = []

# --- CALIBRATION STATE ---
CALIBRATION_PIXEL_POINTS = [] 
CALIBRATION_DATA_MAP = [] 
MANUAL_CALIBRATION_MODE = False

contour_lock = threading.Lock()
socket_lock = threading.Lock()

# Global reference to camera capture
cap = None
running = True

# --- Area preview (density-based) ---
area_preview_points = []
area_preview_lock = threading.Lock()

# --- Flask App for Tablet Interface ---
app = Flask(__name__)

@app.before_request
def log_request_info():
    print(f"\n>>> INCOMING REQUEST: {request.method} {request.path}", flush=True)
    if request.method == 'POST':
        print(f">>> Content-Type: {request.content_type}", flush=True)
        print(f">>> Data: {request.data[:200] if request.data else 'None'}", flush=True)

# HTML Interface that runs on the tablet browser
TABLET_HTML = """
<!DOCTYPE html>
<html>
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0, user-scalable=no">
<meta http-equiv="Cache-Control" content="no-cache, no-store, must-revalidate">
<meta http-equiv="Pragma" content="no-cache">
<meta http-equiv="Expires" content="0">
<style>
body {
    margin: 0;
    background: #000;
    color: #0f0;
    font-family: monospace;
    overflow: hidden;
}
#touchpad {
    position: absolute;
    top: 0;
    left: 0;
    width: 100%;
    height: 100%;
    z-index: 10;
    touch-action: none;
}
#video-background {
    position: absolute;
    width: 100%;
    height: 100%;
    object-fit: contain;
}
.sidebar {
    position: absolute;
    right: 20px;
    top: 20px;
    bottom: 20px;
    display: flex;
    flex-direction: column;
    gap: 15px;
    z-index: 25;
}
.btn {
    width: 120px;
    height: 80px;
    background: rgba(0,0,0,0.85);
    border: 3px solid #0f0;
    display: flex;
    align-items: center;
    justify-content: center;
    border-radius: 15px;
    font-weight: bold;
    font-size: 24px;
    cursor: pointer;
    color: #0f0;
    transition: all 0.2s;
}
.btn:active {
    transform: scale(0.95);
    background: rgba(0,255,0,0.2);
}
.btn-exit {
    border-color: #ff0;
    color: #ff0;
}
.btn-exit:active {
    background: rgba(255,255,0,0.2);
}
.btn-lase {
    border-color: #f00;
    color: #f00;
    font-size: 22px;
}
.btn-lase:active {
    background: rgba(255,0,0,0.2);
}
.slider-container {
    width: 120px;
    background: rgba(0,0,0,0.85);
    border: 3px solid #0f0;
    border-radius: 15px;
    display: flex;
    flex-direction: column;
    align-items: center;
    justify-content: center;
    padding: 15px 0;
    gap: 10px;
}
.slider-label {
    font-size: 18px;
    color: #0f0;
    font-weight: bold;
}
input[type=range][orient=vertical] {
    -webkit-appearance: slider-vertical;
    width: 40px;
    height: 200px;
    cursor: pointer;
}
.slider-value {
    font-size: 28px;
    color: #fff;
    font-weight: bold;
}
</style>
</head>
<body>
<img id="video-background" src="/video_feed">
<div id="touchpad"></div>
<div class="sidebar">
    <div class="btn btn-lase" onclick="cmd('lasing')">LASING</div>
    <div class="slider-container">
        <div class="slider-label">DENSITY</div>
        <input type="range" min="5" max="100" step="5" value="10"
               orient="vertical" oninput="updateDensity(this.value)">
        <div class="slider-value" id="dval">10</div>
    </div>
    <div class="slider-container">
        <div class="slider-label">ON TIME</div>
        <input type="range" min="1" max="500" step="1" value="50" 
               orient="vertical" oninput="updateOnTime(this.value)">
        <div class="slider-value" id="oval">50ms</div>
    </div>
    <div class="slider-container">
        <div class="slider-label">THRESH</div>
        <input type="range" min="0" max="255" value="80" 
               orient="vertical" oninput="updateThresh(this.value)">
        <div class="slider-value" id="tval">80</div>
    </div>
    <div class="btn" onclick="cmd('calibrate')">CALIB</div>
    <div class="btn" onclick="cmd('home')">HOME</div>
    <div class="btn" onclick="cmd('clear')">CLEAR</div>
    <div class="btn btn-exit" onclick="cmd('exit')">EXIT</div>
</div>
<script>
function cmd(c) {
    console.log("Button clicked:", c);
    fetch('/command', {
        method:'POST', 
        headers:{'Content-Type':'application/json'}, 
        body:JSON.stringify({command:c})
    })
    .then(response => response.json())
    .then(data => console.log("Response:", data))
    .catch(error => console.error("Error:", error));
}
function updateOnTime(v) {
    document.getElementById('oval').innerText = v + 'ms';
    fetch('/update_ontime', {
        method:'POST',
        headers:{'Content-Type':'application/json'},
        body:JSON.stringify({ontime:parseInt(v)})
    })
    .then(response => response.json())
    .catch(error => console.error("Error updating ON TIME:", error));
}
function updateThresh(v) {
    document.getElementById('tval').innerText = v;
    fetch('/update_threshold', {
        method:'POST',
        headers:{'Content-Type':'application/json'},
        body:JSON.stringify({threshold:parseInt(v)})
    })
    .then(response => response.json())
    .then(data => console.log("Threshold updated:", data))
    .catch(error => console.error("Error:", error));
}
function updateDensity(v) {
    console.log("Density slider moved to:", v);
    document.getElementById('dval').innerText = v;
    
    const url = '/update_density';
    console.log("Fetching URL:", url);
    
    fetch(url, {
        method:'POST',
        headers:{'Content-Type':'application/json'},
        body:JSON.stringify({density:parseInt(v)})
    })
    .then(response => {
        console.log("Density response status:", response.status);
        console.log("Density response headers:", response.headers);
        if (!response.ok) {
            return response.text().then(text => {
                console.error("Response body:", text);
                throw new Error(`HTTP ${response.status}: ${text}`);
            });
        }
        return response.json();
    })
    .then(data => {
        console.log("Density updated successfully:", data);
    })
    .catch(error => {
        console.error("Density update ERROR:", error);
        alert("Density update failed: " + error.message);
    });
}
const tp = document.getElementById('touchpad');
function send(t, cx, cy) {
    const r = tp.getBoundingClientRect();
    const x = Math.round(((cx - r.left)/r.width)*1920);
    const y = Math.round(((cy - r.top)/r.height)*1440);
    fetch('/touch_input', {
        method:'POST',
        headers:{'Content-Type':'application/json'},
        body:JSON.stringify({touching:t, x:x, y:y})
    });
}
tp.addEventListener('touchstart', (e)=>{
    e.preventDefault();
    send(true, e.touches[0].clientX, e.touches[0].clientY);
});
tp.addEventListener('touchmove', (e)=>{
    e.preventDefault();
    send(true, e.touches[0].clientX, e.touches[0].clientY);
});
tp.addEventListener('touchend', ()=>{
    send(false, 0, 0);
});

// Test connection on load
window.addEventListener('load', function() {
    console.log("Page loaded, testing connection...");
    fetch('/test')
        .then(response => response.json())
        .then(data => console.log("Server test:", data))
        .catch(error => console.error("Server test failed:", error));
});
</script>
</body>
</html>
"""

def generate_web_stream():
    global output_frame
    last_frame_time = 0
    fps_limit = 1/20  # Limit stream to 20 FPS
    
    while True:
        current_time = time.time()
        if current_time - last_frame_time < fps_limit:
            time.sleep(0.03)
            continue
        
        with video_lock:
            if output_frame is None:
                continue
            encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), 30]
            flag, encodedImage = cv2.imencode(".jpg", output_frame, encode_param)
            if not flag:
                continue
        
        last_frame_time = current_time
        yield(b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' + 
              bytearray(encodedImage) + b'\r\n')

@app.route('/')
def tablet_interface():
    print("[FLASK] Tablet accessed main page", flush=True)
    return TABLET_HTML

@app.route("/video_feed")
def video_feed():
    return Response(generate_web_stream(), 
                    mimetype="multipart/x-mixed-replace; boundary=frame")

@app.route('/test', methods=['GET'])
def test_endpoint():
    print("[TEST] Test endpoint accessed!", flush=True)
    return jsonify({'status': 'ok', 'message': 'Server is working!'})

@app.route('/command', methods=['POST'])
def handle_ui_command():
    global running, CALIBRATION_PIXEL_POINTS, detected_contours, daq_socket
    global finished_polys, last_active_contour, current, contours, AREA_STEP
    global aiming_thread_active
    
    try:
        data = request.json
        cmd = data.get('command')
        print(f"\n==== [COMMAND] {cmd} ====", flush=True)
    except Exception as e:
        print(f"ERROR parsing command: {e}", flush=True)
        return jsonify({'status': 'error', 'message': str(e)})
    
    if cmd == 'lasing':
        if last_active_contour is not None:
            # STOP aiming
            with aiming_lock:
                aiming_thread_active = False
            time.sleep(0.1)
            
            area_pts = get_area_points(last_active_contour, AREA_STEP)
            
            with socket_lock:
                # ARM
                daq_socket.sendall(f"SET_ONTIME,{LASER_ON_TIME}\n".encode('ascii'))
                time.sleep(0.05)
                
                # FIRE
                send_batch_to_server(area_pts, daq_socket)
                time.sleep(0.05)
                
                # DISARM
                daq_socket.sendall(b"SET_ONTIME,0\n")
            
            print(f"[LASING] Fired {len(area_pts)} points.")
            
            # RESTART aiming
            time.sleep(0.1)
            with aiming_lock:
                aiming_thread_active = True
        else:
            print("[LASING] No contour selected.")
                
    elif cmd == 'home':
        print(f"\n[HOME] Button pressed", flush=True)
        try:
            if daq_socket:
                with socket_lock:
                    daq_socket.sendall(f"{HOME_X},{HOME_Y}\n".encode('ascii'))
                    print(f"[HOME] Sent galvo to HOME position: {HOME_X}, {HOME_Y}", flush=True)
            else:
                print("[HOME] Error: DAQ socket not connected", flush=True)
        except Exception as e:
            print(f"[HOME] Error: {e}", flush=True)
    
    elif cmd == 'clear':
        print(f"\n[CLEAR] Button pressed", flush=True)
        try:
            with aiming_lock:
                aiming_thread_active = False
            time.sleep(0.1)
            with contour_lock:
                finished_polys.clear()
                contours.clear()
                current = []
                last_active_contour = None
            print("[CLEAR] All drawings cleared", flush=True)
        except Exception as e:
            print(f"[CLEAR] Error: {e}", flush=True)
    
    elif cmd == 'calibrate':
        print("[CALIBRATE] Starting calibration process...")
        threading.Thread(target=calibrate_galvo_camera, daemon=True).start()
    
    elif cmd == 'exit':
        running = False
        print("[EXIT] Shutting down...")
    
    return jsonify({'status': 'ok'})

@app.route('/touch_input', methods=['POST'])
def receive_touch():
    data = request.json
    touching, x, y = data.get('touching'), data.get('x'), data.get('y')
    
    if touching:
        if not touch_active:
            handle_touch_down(x, y)
        else:
            handle_touch_move(x, y)
    elif touch_active:
        handle_touch_up(x, y)
    
    return jsonify({'status': 'ok'})

@app.route('/update_threshold', methods=['POST'])
def update_threshold():
    global GLOBAL_GRAYSCALE_THRESHOLD
    sys.stdout.write("\n[THRESHOLD] Route accessed!\n")
    sys.stdout.flush()
    try:
        val = request.json.get('threshold', 80)
        GLOBAL_GRAYSCALE_THRESHOLD = int(val)
        sys.stdout.write(f"[THRESH] Value set to: {GLOBAL_GRAYSCALE_THRESHOLD}\n")
        sys.stdout.flush()
        return jsonify({'status': 'ok', 'threshold': GLOBAL_GRAYSCALE_THRESHOLD})
    except Exception as e:
        sys.stdout.write(f"[THRESH] ERROR: {e}\n")
        sys.stdout.flush()
        return jsonify({'status': 'error', 'message': str(e)})

@app.route('/update_ontime', methods=['POST'])
def update_ontime():
    global LASER_ON_TIME
    try:
        data = request.get_json()
        val = data.get('ontime', 50)
        LASER_ON_TIME = int(val)
        # We send this value to the listener by prepending it to the batch or command
        print(f"[LASER] On-time set to: {LASER_ON_TIME}ms")
        return jsonify({'status': 'ok', 'ontime': LASER_ON_TIME})
    except Exception as e:
        return jsonify({'status': 'error', 'message': str(e)}), 500

@app.route('/update_density', methods=['POST'])
def update_density():
    print(">>> DENSITY ENDPOINT HIT", flush=True)
    data = request.get_json()
    print(">>> RAW DATA:", data, flush=True)
    global AREA_STEP
    sys.stdout.write("\n[DENSITY] Route accessed!\n")
    sys.stdout.flush()
    try:
        data = request.get_json()
        if not data:
            return jsonify({'status': 'error', 'message': 'No JSON data'}), 400
            
        val = data.get('density', 10)
        AREA_STEP = int(np.clip(val, 5, 100))
        AREA_STEP = int(np.clip(val, 5, 100))
        update_area_preview()
        sys.stdout.write(f"[DENSITY] AREA_STEP set to: {AREA_STEP} pixels\n")
        sys.stdout.flush()
        return jsonify({'status': 'ok', 'area_step': AREA_STEP})
    except Exception as e:
        sys.stdout.write(f"[DENSITY] ERROR: {e}\n")
        sys.stdout.flush()
        return jsonify({'status': 'error', 'message': str(e)}), 500

def run_flask_server():
    print("\n" + "="*60, flush=True)
    print("FLASK SERVER STARTING", flush=True)
    print(f"Server IP: {SERVER_IP}", flush=True)
    print(f"Server Port: {TABLET_SERVER_PORT}", flush=True)
    print(f"Access from tablet: http://{SERVER_IP}:{TABLET_SERVER_PORT}", flush=True)
    print("\nRegistered routes:", flush=True)
    for rule in app.url_map.iter_rules():
        print(f"  {rule.endpoint}: {rule.rule} {list(rule.methods)}", flush=True)
    print("="*60 + "\n", flush=True)
    app.run(host=SERVER_IP, port=TABLET_SERVER_PORT, debug=False, threaded=True)


# --------------------------------------------
# HSV LASER DETECTION
# --------------------------------------------
def find_laser_spot_hsv(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    l1, u1 = np.array([0, 50, 50]), np.array([10, 255, 255])
    l2, u2 = np.array([160, 50, 50]), np.array([180, 255, 255])
    mask = cv2.addWeighted(cv2.inRange(hsv, l1, u1), 1.0, cv2.inRange(hsv, l2, u2), 1.0, 0)
    cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if cnts:
        c = max(cnts, key=cv2.contourArea)
        if cv2.contourArea(c) > 2:
            M = cv2.moments(c)
            if M["m00"] != 0:
                return (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
    return None

# --------------------------------------------
# AREA & BOUNDARY PROCESSING
# --------------------------------------------
def get_area_points(polygon, step):
    """Generate grid points inside polygon with given step size"""
    pts_to_send = []
    x, y, w, h = cv2.boundingRect(polygon)
    for j in range(y, y + h, step):
        row = []
        for i in range(x, x + w, step):
            if cv2.pointPolygonTest(polygon, (float(i), float(j)), False) >= 0:
                row.append((i, j))
        pts_to_send.extend(row)
    return pts_to_send

def update_area_preview():
    """Recompute area fill preview using latest AREA_STEP"""
    global area_preview_points, last_active_contour, AREA_STEP

    with area_preview_lock:
        area_preview_points.clear()

        if last_active_contour is None:
            return

        area_preview_points = get_area_points(last_active_contour, AREA_STEP)
        print(f"[PREVIEW] Updated area preview: {len(area_preview_points)} points (AREA_STEP={AREA_STEP})", flush=True)

def distance(p1, p2):
    return math.hypot(int(p1[0]) - int(p2[0]), int(p1[1]) - int(p2[1]))

def get_sampled_contour(contour, min_jump_dist):
    """Sample contour boundary points with given minimum distance"""
    points = np.array(contour).reshape(-1, 2)
    if not points.any():
        return []
    
    sampled = [tuple(points[0])]
    last_point = points[0]
    
    for pt in points[1:]:
        if distance(last_point, pt) >= min_jump_dist:
            sampled.append(tuple(pt))
            last_point = pt
    
    return sampled

def send_batch_to_server(points_list, daq_socket):
    if not daq_socket or not points_list:
        return
    
    with socket_lock:
        try:
            daq_socket.sendall(b"BATCH_START\n")
            data_string = ";".join(["{0},{1}".format(int(x), int(y)) for x, y in points_list])
            daq_socket.sendall(data_string.encode('ascii') + b"\n")
        except Exception as e:
            print("Batch error: {0}".format(e))

def aiming_loop():
    """Background thread that continuously aims at contour boundary (non-blocking)"""
    global aiming_contour, aiming_thread_active
    
    last_boundary_pts = []
    current_pt_index = 0
    
    while running:
        time.sleep(0.01)  # 100 Hz - faster, non-blocking checks
        
        # Read aiming state (quick lock/unlock)
        with aiming_lock:
            if not aiming_thread_active or aiming_contour is None:
                last_boundary_pts = []
                current_pt_index = 0
                continue
            contour = aiming_contour.copy()  # Copy to avoid holding lock
        
        # Compute boundary points (only if contour changed)
        boundary_pts = get_sampled_contour(contour, MIN_JUMP_DIST)
        
        if len(boundary_pts) == 0:
            continue
        
        # If boundary changed, restart from beginning
        if len(boundary_pts) != len(last_boundary_pts):
            last_boundary_pts = boundary_pts
            current_pt_index = 0
        
        # Send ONE point per iteration (non-blocking)
        if daq_socket and len(boundary_pts) > 0:
            try:
                pt = boundary_pts[current_pt_index % len(boundary_pts)]
                
                # NON-BLOCKING: Try to send, but don't wait
                try:
                    with socket_lock:
                        daq_socket.sendall(f"{int(pt[0])},{int(pt[1])}\n".encode('ascii'))
                except BlockingIOError:
                    # Socket buffer full, skip this point
                    pass
                
                current_pt_index += 1
                
            except Exception as e:
                print(f"[AIMING] Error: {e}")
                
def find_dark_spots(frame):
    global GLOBAL_GRAYSCALE_THRESHOLD, detected_contours
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(gray, GLOBAL_GRAYSCALE_THRESHOLD, 255, cv2.THRESH_BINARY_INV)
    kernel = np.ones((3, 3), np.uint8)
    mask = cv2.dilate(cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel), kernel, iterations=1)
    detected, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    detected_contours = [c for c in detected if MIN_AREA < cv2.contourArea(c) < MAX_AREA]
    return detected_contours

# --------------------------------------------
# TOUCH INPUT HANDLER
# --------------------------------------------
def calculate_path_length(points):
    if len(points) < 2:
        return 0
    total = 0
    for i in range(1, len(points)):
        total += distance(points[i-1], points[i])
    return total

def handle_touch_down(x, y):
    global drawing, current, touch_active, touch_start_pos, touch_current_path
    global last_active_contour, aiming_thread_active
    
    if MANUAL_CALIBRATION_MODE:
        return
    
    pt = (int(x), int(y))
    
    # Check if clicking a detected dark spot
    for detected_c in detected_contours:
        if cv2.pointPolygonTest(detected_c, pt, False) >= 0:
            poly_arr = np.array(detected_c, dtype=np.int32).reshape(-1, 2)
            finished_polys.append(poly_arr)
            last_active_contour = poly_arr
            update_area_preview()
            
            # START AIMING (no firing yet)
            with aiming_lock:
                aiming_contour = poly_arr
                aiming_thread_active = True
            return
    
    # Start new manual drawing
    touch_active = True
    touch_start_pos, touch_current_path = pt, [pt]
    drawing = True
    current = [pt]


def handle_touch_move(x, y):
    global current, touch_current_path
    
    if not touch_active or MANUAL_CALIBRATION_MODE:
        return
    
    pt = (int(x), int(y))
    touch_current_path.append(pt)
    
    # Just add to current for visual feedback
    if not current or distance(current[-1], pt) >= 15:
        with contour_lock:
            current.append(pt)

def handle_touch_up(x, y):
    global drawing, current, touch_active, last_active_contour, aiming_thread_active
    
    if not touch_active or MANUAL_CALIBRATION_MODE:
        return
    
    touch_active = False
    drawing = False
    
    path_len = calculate_path_length(touch_current_path)
    
    if path_len < SINGLE_POINT_THRESHOLD:
        # Single tap
        poly_arr = np.array([touch_start_pos], dtype=np.int32)
        last_active_contour = poly_arr
        update_area_preview()
        
        with aiming_lock:
            aiming_contour = poly_arr
            aiming_thread_active = True
        print(f"[TOUCH] Single tap at: {touch_start_pos}", flush=True)

    elif len(current) > 5:
        dist_to_start = distance(current[0], current[-1])
        
        if dist_to_start < CLOSED_THRESHOLD:
            current.append(current[0])  # Close the shape
        
        poly_arr = np.array(current, dtype=np.int32)
        finished_polys.append(poly_arr)
        last_active_contour = poly_arr
        update_area_preview()
        
        # START AIMING at the drawn contour
        with aiming_lock:
            aiming_contour = poly_arr
            aiming_thread_active = True
        
        print(f"[TOUCH] Contour drawn with {len(current)} points, aiming...", flush=True)
    
    current = []
    touch_current_path = []  # Reset touch path

# --------------------------------------------
# LIVE FEEDBACK CALIBRATION
# --------------------------------------------
def calibrate_galvo_camera():
    global daq_socket, MANUAL_CALIBRATION_MODE, CALIBRATION_DATA_MAP, CALIBRATION_PIXEL_POINTS, cap
    global aiming_thread_active
    
    if not daq_socket:
        print("ERROR: Not connected to DAQ server.")
        return False
    
    if cap is None:
        print("ERROR: Camera not initialized.")
        return False
    
    # STOP aiming during calibration
    with aiming_lock:
        aiming_thread_active = False
    time.sleep(0.1)
    
    MANUAL_CALIBRATION_MODE = True
    CALIBRATION_DATA_MAP.clear()
    CALIBRATION_PIXEL_POINTS.clear()
    
    with socket_lock:
        daq_socket.sendall(b"CALIBRATION_START\n")
        time.sleep(1)

        cx, cy = (GALVO_MAX_X + GALVO_MIN_X)//2, (GALVO_MAX_Y + GALVO_MIN_Y)//2
        xs = np.linspace(GALVO_MIN_X, GALVO_MAX_X, CROSS_SIZE, dtype=np.int32)
        ys = np.linspace(GALVO_MIN_Y, GALVO_MAX_Y, CROSS_SIZE, dtype=np.int32)
        gal_pts = [(cx, int(y)) for y in ys] + [(int(x), cy) for x in xs if int(x) != cx]
        
        print(f"Moving to {len(gal_pts)} positions...")

        for i, (x_dac, y_dac) in enumerate(gal_pts):
            if not MANUAL_CALIBRATION_MODE:
                break
            
            daq_socket.sendall("{0},{1}\n".format(int(x_dac), int(y_dac)).encode('ascii'))
            time.sleep(0.6)
            
            start_search = time.time()
            found_pt = (0, 0)
            
            while (time.time() - start_search) < 2.0:
                ret, frame = cap.read()
                if not ret:
                    break
                
                spot = find_laser_spot_hsv(frame)
                if spot:
                    found_pt = spot
                    CALIBRATION_PIXEL_POINTS.append(spot)
                    print(f"[CALIB] Found spot {i+1}/{len(gal_pts)} at: {spot}")
                    break
                
                time.sleep(0.01)

            CALIBRATION_DATA_MAP.append(found_pt)

        daq_socket.sendall(b"PIXEL_DUMP_START\n")
        time.sleep(0.2)
        
        for px, py in CALIBRATION_DATA_MAP:
            daq_socket.sendall("{0},{1}\n".format(int(px), int(py)).encode('ascii'))
            time.sleep(0.05)
        
        daq_socket.sendall(b"PIXEL_DUMP_END\n")
        print("Calibration complete. Table updated.")
    
    MANUAL_CALIBRATION_MODE = False
    return True

# -------------------------------------------- 
# MAIN EXECUTION
# --------------------------------------------
try:
    daq_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    daq_socket.connect((HOST, PORT))
    print("Connected to DAQ server")
except:
    daq_socket = None
    print("Warning: Could not connect to DAQ server")

aiming_thread = threading.Thread(target=aiming_loop, daemon=True)
aiming_thread.start()
print("[MAIN] Aiming thread started")

flask_thread = threading.Thread(target=run_flask_server, daemon=True)
flask_thread.start()

cap = cv2.VideoCapture(1)
# Set resolution to 1920x1440
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1440)

# cv2.namedWindow(WINDOW_NAME, cv2.WINDOW_NORMAL)

while running:
    ret, frame = cap.read()
    if not ret:
        break

    if not MANUAL_CALIBRATION_MODE:
        # Detect dark spots (red outlines)
        detected_contours = find_dark_spots(frame)
        
        overlay = frame.copy()
        
        # Draw all persistent closed/selected contours in Green (50% alpha)
        for poly in finished_polys:
            cv2.fillPoly(overlay, [poly], (0, 255, 0))
            cv2.polylines(frame, [poly], True, (0, 255, 0), 2)
        
        cv2.addWeighted(overlay, 0.5, frame, 0.5, 0, frame)

        # Draw active drawing lines (magenta)
        with contour_lock:
            for cnt in contours:
                if len(cnt) >= 2:
                    pts = np.array(cnt, np.int32).reshape((-1, 1, 2))
                    cv2.polylines(frame, [pts], False, (255, 0, 255), 3)

        # Draw calibration crosses (blue)
        for pt in CALIBRATION_PIXEL_POINTS:
            cv2.drawMarker(frame, pt, (255, 0, 0), cv2.MARKER_CROSS, 20, 3)
        
        # Draw detected dark spots (red outlines)
        cv2.drawContours(frame, detected_contours, -1, (0, 0, 255), 2)
    
    # Draw calibration crosses during calibration mode
    if MANUAL_CALIBRATION_MODE:
        for pt in CALIBRATION_PIXEL_POINTS:
            cv2.drawMarker(frame, pt, (255, 0, 0), cv2.MARKER_CROSS, 20, 3)
    with area_preview_lock:
        for (x, y) in area_preview_points:
            cv2.circle(frame, (int(x), int(y)), 5, (0, 0, 255), -1)

    with video_lock:
        output_frame = frame.copy()
    
    cv2.imshow(WINDOW_NAME, frame)
    
    # Keyboard shortcuts still available
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'): 
        running = False
    elif key == ord('c'): 
        with contour_lock:
            finished_polys.clear()
            contours.clear()
            last_active_contour = None
        print("Drawings cleared.")
    elif key == ord('a'): 
        threading.Thread(target=calibrate_galvo_camera, daemon=True).start()
    elif key == ord('l'):
        if last_active_contour is not None:
            print(f"[LASING] Starting area fill with AREA_STEP={AREA_STEP}...")
            with area_preview_lock:
                area_pts = list(area_preview_points)            
            send_batch_to_server(area_pts, daq_socket)
            print(f"[LASING] Sent {len(area_pts)} points")

if daq_socket:
    daq_socket.close()
cap.release()
cv2.destroyAllWindows()

Connected to DAQ server
[MAIN] Aiming thread started

FLASK SERVER STARTING
Server IP: 0.0.0.0
Server Port: 8081
Access from tablet: http://0.0.0.0:8081

Registered routes:
  static: /static/<path:filename> ['HEAD', 'OPTIONS', 'GET']
  tablet_interface: / ['HEAD', 'OPTIONS', 'GET']
  video_feed: /video_feed ['HEAD', 'OPTIONS', 'GET']
  test_endpoint: /test ['HEAD', 'OPTIONS', 'GET']
  handle_ui_command: /command ['POST', 'OPTIONS']
  receive_touch: /touch_input ['POST', 'OPTIONS']
  update_threshold: /update_threshold ['POST', 'OPTIONS']
  update_ontime: /update_ontime ['POST', 'OPTIONS']
  update_density: /update_density ['POST', 'OPTIONS']

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8081
 * Running on http://192.168.51.107:8081
Press CTRL+C to quit


KeyboardInterrupt: 


>>> INCOMING REQUEST: GET /
[FLASK] Tablet accessed main page


127.0.0.1 - - [30/Dec/2025 13:41:18] "GET / HTTP/1.1" 200 -


...


>>> INCOMING REQUEST: POST /touch_input
>>> Content-Type: application/json
>>> Data: b'{"touching":true,"x":933,"y":968}'


169.254.0.1 - - [30/Dec/2025 13:56:30] "POST /touch_input HTTP/1.1" 200 -



>>> INCOMING REQUEST: POST /touch_input
>>> Content-Type: application/json
>>> Data: b'{"touching":false,"x":0,"y":0}'
[PREVIEW] Updated area preview: 544 points (AREA_STEP=10)
[TOUCH] Closed contour created and saved for LASING
[TOUCH] Contour has 36 points


169.254.0.1 - - [30/Dec/2025 13:56:30] "POST /touch_input HTTP/1.1" 200 -



>>> INCOMING REQUEST: POST /command
>>> Content-Type: application/json
>>> Data: b'{"command":"lasing"}'

==== [COMMAND] lasing ====


In [ ]:
##################################### Wide Camera Control (1920x1440 with ELO Touch)  
# [compatible with (Server)_daq_listener_calibration+batching_wide_camera_rev4_Claude]#####################################


import cv2
import numpy as np
import math
import socket
import threading
import time
import sys
from flask import Flask, jsonify, request, Response

# For Jupyter/IPython compatibility
try:
    sys.stdout.reconfigure(line_buffering=True)
    sys.stderr.reconfigure(line_buffering=True)
except AttributeError:
    # Running in Jupyter/IPython - output is already unbuffered
    pass

# --- DAC/Voltage Configuration ---
DAC_MAX = 1920
AO_RANGE = 5.0

# --- Network Configuration ---
HOST = '127.0.0.1'
PORT = 65432
daq_socket = None

# Web server for tablet
TABLET_SERVER_PORT = 8082
SERVER_IP = '0.0.0.0'  # Bind to all network interfaces

# --- Drawing Configuration ---
MIN_DIST = 2              
MIN_JUMP_DIST = 10        
CLOSED_THRESHOLD = 100    
AREA_STEP = 10  # This will be updated by the density slider
SINGLE_POINT_THRESHOLD = 20

# --- Dark Spot Detection Configuration ---
GLOBAL_GRAYSCALE_THRESHOLD = 80
WINDOW_NAME = "Laser Controller"
MIN_AREA = 500
MAX_AREA = 500000

# --- Galvo Configuration (adjusted for 1920x1440) ---
HOME_X = 960              # Center X for 1920x1440
HOME_Y = 720              # Center Y for 1920x1440
GALVO_MIN_X = 125         # Scaled from 250
GALVO_MAX_X = 1750        # Scaled from 3500
GALVO_MIN_Y = 0
GALVO_MAX_Y = 1333        # Scaled from 2000
CROSS_SIZE = 10           

# --- Global State ---
contours = []             
finished_polys = []       
last_active_contour = None 
detected_contours = []
current = []
drawing = False 

# Video Streaming State
video_lock = threading.Lock()
output_frame = None

# Touch state
touch_active = False
touch_start_pos = None
touch_current_path = []

# --- CALIBRATION STATE ---
CALIBRATION_PIXEL_POINTS = [] 
CALIBRATION_DATA_MAP = [] 
MANUAL_CALIBRATION_MODE = False

contour_lock = threading.Lock()
socket_lock = threading.Lock()

# Global reference to camera capture
cap = None
running = True

# --- Area preview (density-based) ---
area_preview_points = []
area_preview_lock = threading.Lock()

# --- Flask App for Tablet Interface ---
app = Flask(__name__)

@app.before_request
def log_request_info():
    print(f"\n>>> INCOMING REQUEST: {request.method} {request.path}", flush=True)
    if request.method == 'POST':
        print(f">>> Content-Type: {request.content_type}", flush=True)
        print(f">>> Data: {request.data[:200] if request.data else 'None'}", flush=True)

# HTML Interface that runs on the tablet browser
TABLET_HTML = """
<!DOCTYPE html>
<html>
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0, user-scalable=no">
<meta http-equiv="Cache-Control" content="no-cache, no-store, must-revalidate">
<meta http-equiv="Pragma" content="no-cache">
<meta http-equiv="Expires" content="0">
<style>
body {
    margin: 0;
    background: #000;
    color: #0f0;
    font-family: monospace;
    overflow: hidden;
}
#touchpad {
    position: absolute;
    top: 0;
    left: 0;
    width: 100%;
    height: 100%;
    z-index: 10;
    touch-action: none;
}
#video-background {
    position: absolute;
    width: 100%;
    height: 100%;
    object-fit: contain;
}
.sidebar {
    position: absolute;
    right: 20px;
    top: 20px;
    bottom: 20px;
    display: flex;
    flex-direction: column;
    gap: 15px;
    z-index: 25;
}
.btn {
    width: 120px;
    height: 80px;
    background: rgba(0,0,0,0.85);
    border: 3px solid #0f0;
    display: flex;
    align-items: center;
    justify-content: center;
    border-radius: 15px;
    font-weight: bold;
    font-size: 24px;
    cursor: pointer;
    color: #0f0;
    transition: all 0.2s;
}
.btn:active {
    transform: scale(0.95);
    background: rgba(0,255,0,0.2);
}
.btn-exit {
    border-color: #ff0;
    color: #ff0;
}
.btn-exit:active {
    background: rgba(255,255,0,0.2);
}
.btn-lase {
    border-color: #f00;
    color: #f00;
    font-size: 22px;
}
.btn-lase:active {
    background: rgba(255,0,0,0.2);
}
.slider-container {
    width: 120px;
    background: rgba(0,0,0,0.85);
    border: 3px solid #0f0;
    border-radius: 15px;
    display: flex;
    flex-direction: column;
    align-items: center;
    justify-content: center;
    padding: 15px 0;
    gap: 10px;
}
.slider-label {
    font-size: 18px;
    color: #0f0;
    font-weight: bold;
}
input[type=range][orient=vertical] {
    -webkit-appearance: slider-vertical;
    width: 40px;
    height: 200px;
    cursor: pointer;
}
.slider-value {
    font-size: 28px;
    color: #fff;
    font-weight: bold;
}
</style>
</head>
<body>
<img id="video-background" src="/video_feed">
<div id="touchpad"></div>
<div class="sidebar">
    <div class="btn btn-lase" onclick="cmd('lasing')">LASING</div>
    <div class="slider-container">
        <div class="slider-label">DENSITY</div>
        <input type="range" min="5" max="100" step="5" value="10"
               orient="vertical" oninput="updateDensity(this.value)">
        <div class="slider-value" id="dval">10</div>
    </div>
    <div class="slider-container">
        <div class="slider-label">THRESH</div>
        <input type="range" min="0" max="255" value="80" 
               orient="vertical" oninput="updateThresh(this.value)">
        <div class="slider-value" id="tval">80</div>
    </div>
    <div class="btn" onclick="cmd('calibrate')">CALIB</div>
    <div class="btn" onclick="cmd('home')">HOME</div>
    <div class="btn" onclick="cmd('clear')">CLEAR</div>
    <div class="btn btn-exit" onclick="cmd('exit')">EXIT</div>
</div>
<script>
function cmd(c) {
    console.log("Button clicked:", c);
    fetch('/command', {
        method:'POST', 
        headers:{'Content-Type':'application/json'}, 
        body:JSON.stringify({command:c})
    })
    .then(response => response.json())
    .then(data => console.log("Response:", data))
    .catch(error => console.error("Error:", error));
}
function updateThresh(v) {
    document.getElementById('tval').innerText = v;
    fetch('/update_threshold', {
        method:'POST',
        headers:{'Content-Type':'application/json'},
        body:JSON.stringify({threshold:parseInt(v)})
    })
    .then(response => response.json())
    .then(data => console.log("Threshold updated:", data))
    .catch(error => console.error("Error:", error));
}
function updateDensity(v) {
    console.log("Density slider moved to:", v);
    document.getElementById('dval').innerText = v;
    
    const url = '/update_density';
    console.log("Fetching URL:", url);
    
    fetch(url, {
        method:'POST',
        headers:{'Content-Type':'application/json'},
        body:JSON.stringify({density:parseInt(v)})
    })
    .then(response => {
        console.log("Density response status:", response.status);
        console.log("Density response headers:", response.headers);
        if (!response.ok) {
            return response.text().then(text => {
                console.error("Response body:", text);
                throw new Error(`HTTP ${response.status}: ${text}`);
            });
        }
        return response.json();
    })
    .then(data => {
        console.log("Density updated successfully:", data);
    })
    .catch(error => {
        console.error("Density update ERROR:", error);
        alert("Density update failed: " + error.message);
    });
}
const tp = document.getElementById('touchpad');
function send(t, cx, cy) {
    const r = tp.getBoundingClientRect();
    const x = Math.round(((cx - r.left)/r.width)*1920);
    const y = Math.round(((cy - r.top)/r.height)*1440);
    fetch('/touch_input', {
        method:'POST',
        headers:{'Content-Type':'application/json'},
        body:JSON.stringify({touching:t, x:x, y:y})
    });
}
tp.addEventListener('touchstart', (e)=>{
    e.preventDefault();
    send(true, e.touches[0].clientX, e.touches[0].clientY);
});
tp.addEventListener('touchmove', (e)=>{
    e.preventDefault();
    send(true, e.touches[0].clientX, e.touches[0].clientY);
});
tp.addEventListener('touchend', ()=>{
    send(false, 0, 0);
});

// Test connection on load
window.addEventListener('load', function() {
    console.log("Page loaded, testing connection...");
    fetch('/test')
        .then(response => response.json())
        .then(data => console.log("Server test:", data))
        .catch(error => console.error("Server test failed:", error));
});
</script>
</body>
</html>
"""

def generate_web_stream():
    global output_frame
    last_frame_time = 0
    fps_limit = 1/20  # Limit stream to 20 FPS
    
    while True:
        current_time = time.time()
        if current_time - last_frame_time < fps_limit:
            time.sleep(0.03)
            continue
        
        with video_lock:
            if output_frame is None:
                continue
            encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), 30]
            flag, encodedImage = cv2.imencode(".jpg", output_frame, encode_param)
            if not flag:
                continue
        
        last_frame_time = current_time
        yield(b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' + 
              bytearray(encodedImage) + b'\r\n')

@app.route('/')
def tablet_interface():
    print("[FLASK] Tablet accessed main page", flush=True)
    return TABLET_HTML

@app.route("/video_feed")
def video_feed():
    return Response(generate_web_stream(), 
                    mimetype="multipart/x-mixed-replace; boundary=frame")

@app.route('/test', methods=['GET'])
def test_endpoint():
    print("[TEST] Test endpoint accessed!", flush=True)
    return jsonify({'status': 'ok', 'message': 'Server is working!'})

@app.route('/command', methods=['POST'])
def handle_ui_command():
    global running, CALIBRATION_PIXEL_POINTS, detected_contours, daq_socket
    global finished_polys, last_active_contour, current, contours, AREA_STEP
    
    try:
        data = request.json
        cmd = data.get('command')
        print(f"\n==== [COMMAND RECEIVED] ====", flush=True)
        print(f"Command: {cmd}", flush=True)
        print(f"Current AREA_STEP: {AREA_STEP}", flush=True)
        print(f"============================\n", flush=True)
    except Exception as e:
        print(f"ERROR parsing command: {e}", flush=True)
        return jsonify({'status': 'error', 'message': str(e)})
    
    if cmd == 'lasing':
        if last_active_contour is not None:
            print(f"[LASING] Starting laser operation...", flush=True)
            with area_preview_lock:
                area_pts = list(area_preview_points)
            # Use the new LASING specific sender
            send_lasing_to_server(area_pts, daq_socket)
        else:
            print("[LASING] No active contour selected", flush=True)
            
    elif cmd == 'home':
        print(f"\n[HOME] Button pressed", flush=True)
        try:
            if daq_socket:
                with socket_lock:
                    daq_socket.sendall(f"{HOME_X},{HOME_Y}\n".encode('ascii'))
                    print(f"[HOME] Sent galvo to HOME position: {HOME_X}, {HOME_Y}", flush=True)
            else:
                print("[HOME] Error: DAQ socket not connected", flush=True)
        except Exception as e:
            print(f"[HOME] Error: {e}", flush=True)                             
    
    elif cmd == 'clear':
        print(f"\n[CLEAR] Button pressed", flush=True)
        try:
            with contour_lock:
                finished_polys.clear()
                contours.clear()
                current = []
                last_active_contour = None
            print("[CLEAR] All drawings cleared", flush=True)
        except Exception as e:
            print(f"[CLEAR] Error: {e}", flush=True)
    
    elif cmd == 'calibrate':
        print("[CALIBRATE] Starting calibration process...")
        threading.Thread(target=calibrate_galvo_camera, daemon=True).start()
    
    elif cmd == 'exit':
        running = False
        print("[EXIT] Shutting down...")
    
    return jsonify({'status': 'ok'})

@app.route('/touch_input', methods=['POST'])
def receive_touch():
    data = request.json
    touching, x, y = data.get('touching'), data.get('x'), data.get('y')
    
    if touching:
        if not touch_active:
            handle_touch_down(x, y)
        else:
            handle_touch_move(x, y)
    elif touch_active:
        handle_touch_up(x, y)
    
    return jsonify({'status': 'ok'})

@app.route('/update_threshold', methods=['POST'])
def update_threshold():
    global GLOBAL_GRAYSCALE_THRESHOLD
    sys.stdout.write("\n[THRESHOLD] Route accessed!\n")
    sys.stdout.flush()
    try:
        val = request.json.get('threshold', 80)
        GLOBAL_GRAYSCALE_THRESHOLD = int(val)
        sys.stdout.write(f"[THRESH] Value set to: {GLOBAL_GRAYSCALE_THRESHOLD}\n")
        sys.stdout.flush()
        return jsonify({'status': 'ok', 'threshold': GLOBAL_GRAYSCALE_THRESHOLD})
    except Exception as e:
        sys.stdout.write(f"[THRESH] ERROR: {e}\n")
        sys.stdout.flush()
        return jsonify({'status': 'error', 'message': str(e)})

@app.route('/update_density', methods=['POST'])
def update_density():
    print(">>> DENSITY ENDPOINT HIT", flush=True)
    data = request.get_json()
    print(">>> RAW DATA:", data, flush=True)
    global AREA_STEP
    sys.stdout.write("\n[DENSITY] Route accessed!\n")
    sys.stdout.flush()
    try:
        data = request.get_json()
        if not data:
            return jsonify({'status': 'error', 'message': 'No JSON data'}), 400
            
        val = data.get('density', 10)
        AREA_STEP = int(np.clip(val, 5, 100))
        AREA_STEP = int(np.clip(val, 5, 100))
        update_area_preview()
        sys.stdout.write(f"[DENSITY] AREA_STEP set to: {AREA_STEP} pixels\n")
        sys.stdout.flush()
        return jsonify({'status': 'ok', 'area_step': AREA_STEP})
    except Exception as e:
        sys.stdout.write(f"[DENSITY] ERROR: {e}\n")
        sys.stdout.flush()
        return jsonify({'status': 'error', 'message': str(e)}), 500

def run_flask_server():
    print("\n" + "="*60, flush=True)
    print("FLASK SERVER STARTING", flush=True)
    print(f"Server IP: {SERVER_IP}", flush=True)
    print(f"Server Port: {TABLET_SERVER_PORT}", flush=True)
    print(f"Access from tablet: http://{SERVER_IP}:{TABLET_SERVER_PORT}", flush=True)
    print("\nRegistered routes:", flush=True)
    for rule in app.url_map.iter_rules():
        print(f"  {rule.endpoint}: {rule.rule} {list(rule.methods)}", flush=True)
    print("="*60 + "\n", flush=True)
    app.run(host=SERVER_IP, port=TABLET_SERVER_PORT, debug=False, threaded=True)


# --------------------------------------------
# HSV LASER DETECTION
# --------------------------------------------
def find_laser_spot_hsv(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    l1, u1 = np.array([0, 50, 50]), np.array([10, 255, 255])
    l2, u2 = np.array([160, 50, 50]), np.array([180, 255, 255])
    mask = cv2.addWeighted(cv2.inRange(hsv, l1, u1), 1.0, cv2.inRange(hsv, l2, u2), 1.0, 0)
    cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if cnts:
        c = max(cnts, key=cv2.contourArea)
        if cv2.contourArea(c) > 2:
            M = cv2.moments(c)
            if M["m00"] != 0:
                return (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
    return None

# --------------------------------------------
# AREA & BOUNDARY PROCESSING
# --------------------------------------------
def get_area_points(polygon, step):
    """Generate grid points inside polygon with given step size"""
    pts_to_send = []
    x, y, w, h = cv2.boundingRect(polygon)
    for j in range(y, y + h, step):
        row = []
        for i in range(x, x + w, step):
            if cv2.pointPolygonTest(polygon, (float(i), float(j)), False) >= 0:
                row.append((i, j))
        pts_to_send.extend(row)
    return pts_to_send

def update_area_preview():
    """Recompute area fill preview using latest AREA_STEP"""
    global area_preview_points, last_active_contour, AREA_STEP

    with area_preview_lock:
        area_preview_points.clear()

        if last_active_contour is None:
            return

        area_preview_points = get_area_points(last_active_contour, AREA_STEP)
        print(f"[PREVIEW] Updated area preview: {len(area_preview_points)} points (AREA_STEP={AREA_STEP})", flush=True)

def distance(p1, p2):
    return math.hypot(int(p1[0]) - int(p2[0]), int(p1[1]) - int(p2[1]))

def get_sampled_contour(contour, min_jump_dist):
    """Sample contour boundary points with given minimum distance"""
    points = np.array(contour).reshape(-1, 2)
    if not points.any():
        return []
    
    sampled = [tuple(points[0])]
    last_point = points[0]
    
    for pt in points[1:]:
        if distance(last_point, pt) >= min_jump_dist:
            sampled.append(tuple(pt))
            last_point = pt
    
    return sampled

def send_batch_to_server(points_list, daq_socket):
    if not daq_socket or not points_list:
        return
    
    with socket_lock:
        try:
            daq_socket.sendall(b"BATCH_START\n")
            data_string = ";".join(["{0},{1}".format(int(x), int(y)) for x, y in points_list])
            daq_socket.sendall(data_string.encode('ascii') + b"\n")
        except Exception as e:
            print("Batch error: {0}".format(e))

def find_dark_spots(frame):
    global GLOBAL_GRAYSCALE_THRESHOLD, detected_contours
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(gray, GLOBAL_GRAYSCALE_THRESHOLD, 255, cv2.THRESH_BINARY_INV)
    kernel = np.ones((3, 3), np.uint8)
    mask = cv2.dilate(cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel), kernel, iterations=1)
    detected, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    detected_contours = [c for c in detected if MIN_AREA < cv2.contourArea(c) < MAX_AREA]
    return detected_contours



# --------------------------------------------
# TOUCH INPUT HANDLER
# --------------------------------------------
def calculate_path_length(points):
    if len(points) < 2:
        return 0
    total = 0
    for i in range(1, len(points)):
        total += distance(points[i-1], points[i])
    return total

def handle_touch_down(x, y):
    global drawing, current, touch_active, touch_start_pos, touch_current_path
    global finished_polys, last_active_contour, detected_contours
    
    if MANUAL_CALIBRATION_MODE:
        return
    
    pt = (int(x), int(y))
    
    # Check if clicking algorithm-detected contour (red outlines)
    for detected_c in detected_contours:
        if cv2.pointPolygonTest(detected_c, pt, False) >= 0:
            poly_arr = np.array(detected_c, dtype=np.int32).reshape(-1, 2)
            finished_polys.append(poly_arr)
            last_active_contour = poly_arr
            update_area_preview()
            # Immediately send circumference
            send_batch_to_server(get_sampled_contour(poly_arr, MIN_JUMP_DIST), daq_socket)
            print(f"[TOUCH] Selected detected contour, sent circumference", flush=True)
            return
    
    touch_active = True
    touch_start_pos, touch_current_path = pt, [pt]
    drawing = True
    current = []
    
    with contour_lock:
        contours.clear()
        contours.append(current)
    
    current.append(pt)

def handle_touch_move(x, y):
    global current, touch_current_path
    
    if not touch_active or MANUAL_CALIBRATION_MODE:
        return
    
    pt = (int(x), int(y))
    touch_current_path.append(pt)
    
    if not current or distance(current[-1], pt) >= MIN_DIST:
        with contour_lock:
            current.append(pt)

def handle_touch_up(x, y):
    global drawing, current, touch_active, touch_start_pos, touch_current_path
    global finished_polys, last_active_contour
    
    if not touch_active or MANUAL_CALIBRATION_MODE:
        return
    
    touch_active = False
    drawing = False
    
    path_len = calculate_path_length(touch_current_path)
    
    if path_len < SINGLE_POINT_THRESHOLD:
        # Single point tap - send immediately
        if daq_socket:
            with socket_lock:
                daq_socket.sendall(f"{int(touch_start_pos[0])},{int(touch_start_pos[1])}\n".encode('ascii'))
                print(f"[TOUCH] Single point tap: {touch_start_pos}", flush=True)
    elif len(current) > 5:
        dist_to_start = distance(current[0], current[-1])
        if dist_to_start < CLOSED_THRESHOLD:
            # Closed contour - add to finished polys and send circumference
            current.append(current[0])
            poly_arr = np.array(current, dtype=np.int32)
            finished_polys.append(poly_arr)
            last_active_contour = poly_arr
            update_area_preview()
            send_batch_to_server(get_sampled_contour(poly_arr, MIN_JUMP_DIST), daq_socket)
            print(f"[TOUCH] Closed contour created and saved for LASING", flush=True)
            print(f"[TOUCH] Contour has {len(poly_arr)} points", flush=True)
        else:
            # Open path - just send the path
            send_batch_to_server(get_sampled_contour(current, MIN_JUMP_DIST), daq_socket)
            print(f"[TOUCH] Open path sent (not saved for LASING)", flush=True)
    
    current = []

# --------------------------------------------
# LIVE FEEDBACK CALIBRATION
# --------------------------------------------
def calibrate_galvo_camera():
    global daq_socket, MANUAL_CALIBRATION_MODE, CALIBRATION_DATA_MAP, CALIBRATION_PIXEL_POINTS, cap
    
    if not daq_socket:
        print("ERROR: Not connected to DAQ server.")
        return False
    
    if cap is None:
        print("ERROR: Camera not initialized.")
        return False
    
    MANUAL_CALIBRATION_MODE = True
    CALIBRATION_DATA_MAP.clear()
    CALIBRATION_PIXEL_POINTS.clear()
    
    with socket_lock:
        daq_socket.sendall(b"CALIBRATION_START\n")
        time.sleep(1)

        cx, cy = (GALVO_MAX_X + GALVO_MIN_X)//2, (GALVO_MAX_Y + GALVO_MIN_Y)//2
        xs = np.linspace(GALVO_MIN_X, GALVO_MAX_X, CROSS_SIZE, dtype=np.int32)
        ys = np.linspace(GALVO_MIN_Y, GALVO_MAX_Y, CROSS_SIZE, dtype=np.int32)
        gal_pts = [(cx, int(y)) for y in ys] + [(int(x), cy) for x in xs if int(x) != cx]
        
        print(f"Moving to {len(gal_pts)} positions...")

        for i, (x_dac, y_dac) in enumerate(gal_pts):
            if not MANUAL_CALIBRATION_MODE:
                break
            
            daq_socket.sendall("{0},{1}\n".format(int(x_dac), int(y_dac)).encode('ascii'))
            time.sleep(0.6)
            
            start_search = time.time()
            found_pt = (0, 0)
            
            while (time.time() - start_search) < 2.0:
                ret, frame = cap.read()
                if not ret:
                    break
                
                spot = find_laser_spot_hsv(frame)
                if spot:
                    found_pt = spot
                    CALIBRATION_PIXEL_POINTS.append(spot)
                    print(f"[CALIB] Found spot {i+1}/{len(gal_pts)} at: {spot}")
                    break
                
                time.sleep(0.01)

            CALIBRATION_DATA_MAP.append(found_pt)

        daq_socket.sendall(b"PIXEL_DUMP_START\n")
        time.sleep(0.2)
        
        for px, py in CALIBRATION_DATA_MAP:
            daq_socket.sendall("{0},{1}\n".format(int(px), int(py)).encode('ascii'))
            time.sleep(0.05)
        
        daq_socket.sendall(b"PIXEL_DUMP_END\n")
        print("Calibration complete. Table updated.")
    
    MANUAL_CALIBRATION_MODE = False
    return True

# -------------------------------------------- 
# MAIN EXECUTION
# --------------------------------------------
# --- Cleaned Connection Logic for Client ---
try:
    daq_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    daq_socket.connect((HOST, PORT))
except Exception as e:
    daq_socket = None
    print(f"Warning: Could not connect to DAQ server: {e}")
# Start Flask
flask_thread = threading.Thread(target=run_flask_server, daemon=True)
flask_thread.start()


cap = cv2.VideoCapture(1)
# Set resolution to 1920x1440
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1440)

cv2.namedWindow(WINDOW_NAME, cv2.WINDOW_NORMAL)

while running:
    ret, frame = cap.read()
    if not ret:
        break

    if not MANUAL_CALIBRATION_MODE:
        # Detect dark spots (red outlines)
        detected_contours = find_dark_spots(frame)
        
        overlay = frame.copy()
        
        # Draw all persistent closed/selected contours in Green (50% alpha)
        for poly in finished_polys:
            cv2.fillPoly(overlay, [poly], (0, 255, 0))
            cv2.polylines(frame, [poly], True, (0, 255, 0), 2)
        
        cv2.addWeighted(overlay, 0.5, frame, 0.5, 0, frame)

        # Draw active drawing lines (magenta)
        with contour_lock:
            for cnt in contours:
                if len(cnt) >= 2:
                    pts = np.array(cnt, np.int32).reshape((-1, 1, 2))
                    cv2.polylines(frame, [pts], False, (255, 0, 255), 3)

        # Draw calibration crosses (blue)
        for pt in CALIBRATION_PIXEL_POINTS:
            cv2.drawMarker(frame, pt, (255, 0, 0), cv2.MARKER_CROSS, 20, 3)
        
        # Draw detected dark spots (red outlines)
        cv2.drawContours(frame, detected_contours, -1, (0, 0, 255), 2)
    
    # Draw calibration crosses during calibration mode
    if MANUAL_CALIBRATION_MODE:
        for pt in CALIBRATION_PIXEL_POINTS:
            cv2.drawMarker(frame, pt, (255, 0, 0), cv2.MARKER_CROSS, 20, 3)
    with area_preview_lock:
        for (x, y) in area_preview_points:
            cv2.circle(frame, (int(x), int(y)), 5, (0, 0, 255), -1)

    with video_lock:
        output_frame = frame.copy()
    
    # cv2.imshow(WINDOW_NAME, frame)
    
    # Keyboard shortcuts still available
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'): 
        running = False
    elif key == ord('c'): 
        with contour_lock:
            finished_polys.clear()
            contours.clear()
            last_active_contour = None
        print("Drawings cleared.")
    elif key == ord('a'): 
        threading.Thread(target=calibrate_galvo_camera, daemon=True).start()
    elif key == ord('l'):
        if last_active_contour is not None:
            print(f"[LASING] Starting area fill with AREA_STEP={AREA_STEP}...")
            with area_preview_lock:
                area_pts = list(area_preview_points)            
            send_batch_to_server(area_pts, daq_socket)
            print(f"[LASING] Sent {len(area_pts)} points")

if daq_socket:
    daq_socket.close()
cap.release()
cv2.destroyAllWindows()


FLASK SERVER STARTING
Server IP: 0.0.0.0
Server Port: 8082
Access from tablet: http://0.0.0.0:8082

Registered routes:
  static: /static/<path:filename> ['HEAD', 'OPTIONS', 'GET']
  tablet_interface: / ['HEAD', 'OPTIONS', 'GET']
  video_feed: /video_feed ['HEAD', 'OPTIONS', 'GET']
  test_endpoint: /test ['HEAD', 'OPTIONS', 'GET']
  handle_ui_command: /command ['POST', 'OPTIONS']
  receive_touch: /touch_input ['POST', 'OPTIONS']
  update_threshold: /update_threshold ['POST', 'OPTIONS']
  update_density: /update_density ['POST', 'OPTIONS']

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8082
 * Running on http://192.168.51.107:8082
Press CTRL+C to quit
